In [23]:
import pandas as pd
import pyperclip
import sacrebleu
from pathlib import Path

"""https://arxiv.org/pdf/2404.13813"""

In [2]:
def load_data(dataset_path):
    parent_path = Path(dataset_path).parent

    if not (parent_path / 'train_split.csv').exists():
        df = pd.read_csv(dataset_path)

        df = df.rename(columns={ # TODO: This is bulshit, remove it
            'target': 'mansi',
            'source': 'ru'
        })

        shuffled_df = df.sample(frac=1).reset_index(drop=True)

        total_size = len(shuffled_df)
        test_size = int(total_size * 0.1)
        val_size = int(total_size * 0.1)
        # train_size = total_size - test_size - val_size

        test_df = shuffled_df[:test_size]
        val_df = shuffled_df[test_size: test_size + val_size]
        train_df = shuffled_df[test_size + val_size:]

        train_df.to_csv(parent_path / 'train_split.csv', index=False)
        val_df.to_csv(parent_path / 'val_split.csv', index=False)
        test_df.to_csv(parent_path / 'test_split.csv', index=False)
    else:
        train_df = pd.read_csv(parent_path / 'train_split.csv')
        val_df = pd.read_csv(parent_path / 'val_split.csv')
        test_df = pd.read_csv(parent_path / 'test_split.csv')

    print(f"Train size: {len(train_df)}")
    print(f"Val size: {len(val_df)}")
    print(f"Test size: {len(test_df)}")

    return train_df, val_df, test_df


train_df, val_df, test_df = load_data("data/cleared_v2.csv")

Train size: 63947
Val size: 7993
Test size: 7993


In [21]:
n_train_pairs = 10000
start_pred_pairs_index = 0
end_pred_pairs_index = 50

train_df_pairs = train_df.sample(n_train_pairs)


str_pairs = ""
for index, row in train_df_pairs.iterrows():
    # str_pairs += f"Russian: {row['ru']}\nMansi: {row['mansi']}\n\n"
    str_pairs += f"Mansi: {row['mansi']}\Russian: {row['ru']}\n\n"


prompt = \
"""
You are a translator. Here is pairs of Mansi sentences and their translations in Russian language:
{data}

Your task is to translate this sentences from Mansi to Russian:
{sentences}

Return the translated sentence in Russian language as Python list.
"""
# sample_value = test_df.sample(1)['ru'].iloc[0]
# sample_values = test_df.sample(n_pred_pairs).reset_index()
sample_values = test_df[start_pred_pairs_index:end_pred_pairs_index]

str_ru = ""
tgt_texts = []

for index, row in sample_values.iterrows():
    str_ru += f"{index}. {row['mansi']}\n"
    tgt_texts.append(row['ru'])

sample_value = str_ru


filled_prompt = prompt.format(data=str_pairs, sentences=sample_value)
# print(filled_prompt)

pyperclip.copy(filled_prompt)



In [22]:
mansi_translations = [
    "Хорошая ткань.",
    "Сквозь кедровые ветви зайчик прыгает.",
    "Сколько женщин там было?",
    "Девочка любила маму-папу.",
    "Только Настя сидела близко от него, об этом Силька ей сказала.",
    "Я за столом сижу.",
    "БОБЁР И БОБРЯТА.",
    "Золотой Царь, отец, в котёл вошёл.",
    "Спустя много лет, внук опять встал на ноги.",
    "Затем он перевернул стол.",
    "Отец однажды заболел, куда-то ушел.",
    "Глаза вместе закрывают, на небо смотрит.",
    "В России среди живущих людей 80 процентов считают себя русскими.",
    "Лягушка этим людям сказала: «Добейте Менква, убейте его».",
    "Их повзрослевшая семья в 1958 году в Саранпауль переехала.",
    "Его шею тоже перерублю я.",
    "Старший мужчина пусть не будет.",
    "Лыжи застревают.",
    "На реализацию этой программы деньги выделяет Нефтеюганский район для того чтобы в «Сказке» местное население  проводили соревнования на обласах.",
    "Женщина от проделанной работы очень обрадовалась.",
    "На этом дереве, где ты держишься, лапа-медвежья пусть будет!",
    "Сначала его хорошенько подумай, потом ответь обратно.",
    "Сейчас я про папу и маму напишу.",
    "Жить хочу.",
    "Поэтому она знаменитая, много напрасно не говорит.",
    "Вверх вот он повернулся, сверху идущий Нуми Торум, снизу идущий ёлы Торум, наш народ пусть оберегают, наш народ пусть содержат.",
    "И ты, дочь Похсын Хар Товыль Мольсянын Отыр, ко мне иди!",
    "Евдокия Андреевна во время учёбы «Русско-хантыйский словарь» выпустила.",
    "Борис Хохряков, председатель окружной Думы, первое слово о работе, проделанной за 2015 год, сказал.",
    "Теперь до сюда он.",
]

bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF(word_order=2)  # this metric is called ChrF++

bleu_score = bleu_calc.corpus_score(mansi_translations, [tgt_texts])
chrf_score = chrf_calc.corpus_score(mansi_translations, [tgt_texts])

print(bleu_score)
print(chrf_score)

BLEU = 8.74 42.1/14.0/5.8/1.7 (BP = 1.000 ratio = 1.027 hyp_len = 266 ref_len = 259)
chrF2++ = 37.17
